# TO for the Actuated Spring-Loaded Inverted Pendulum (ASLIP)

In [16]:
import numpy as np 

import matplotlib.pyplot as plt
#mport mpld3
from IPython.display import display
from IPython.display import Math, Latex

from pydrake.all import (BasicVector, ConstantVectorSource, DiagramBuilder,
                         LeafSystem, LogVectorOutput, MeshcatVisualizerCpp,
                         PortDataType, PublishEvent, PyPlotVisualizer,
                         SceneGraph, Simulator, StartMeshcat,
                         UnrestrictedUpdateEvent, WitnessFunctionDirection,
                         namedview)

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator, Box, ne,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, LinearQuadraticRegulator,
                         Linearize, AddMultibodyPlantSceneGraph, VectorSystem, le, ge, OsqpSolver, PrismaticJoint,
                         Cylinder,Box,RigidTransform, CoulombFriction, IllustrationProperties)

from pydrake.examples.compass_gait import CompassGait, CompassGaitGeometry
from pydrake.examples.rimless_wheel import (RimlessWheel, RimlessWheelGeometry,
                                            RimlessWheelParams)

from underactuated import running_as_notebook
from underactuated.jupyter import AdvanceToAndVisualize


# SLIP Dynamics Model

In [3]:
class SLIP_Model():
    def __init__(self):
        
        #SLIP model paramters 

        self.k = 1000 #spring constant 
        self.l_rest = 1 #undeformed leg length
        self.m = 80 #point mass 

        self.g = 9.81

        self.min_leg_torq = -5

        self.max_leg_torq = 5

        self.collocation_nodes_per_contact = 50

        # self.q = np.empty((1,4)) #state vector x,z,l,theta
        # self.qd = np.empty((1,4))
        # self.qdd = np.empty((1,4))

    def manipulator_equations_ground(self, q, qdot, qddot, u):
        """
        Checks if the given state [pos, vel, accel, forcing (inputs) are dynamically feasible]
        Inputs: State
        Outputs: Manipulator Equation Sum (must be zero)
        """
        
        ## Stance Mode Manipulator Equations ##
        #states are in order of x,z,l,theta 
        #x_d, z_d, l_d, theta_d
        # x_dd, z_dd, l_dd, theta_dd

        assert q.size == 4
        assert qdot.size == 4
        assert qddot.size == 4
        assert u.size == 1

        l = q[2]
        l_d = qdot[2]
        l_dd = qddot[2]

        theta = q[3]
        theta_d = qdot[3]
        theta_dd = qddot[3]

        g = self.g
        m = self.m 
        s1 = np.sin(theta)
        c1 = np.cos(theta)
        k = self.k 
        l0 = self.l_rest 

        a = m*(-g*s1 + l*theta_dd + 2*l_d*theta_d)*l - u[0]
        b = g*m*c1 - k*(l0 - l)  - m*l*theta_d**2 + m*l_dd 

        return a+b 

    def manipulator_equations_air(self,qddot):
        """
        Checks if the given swing/flight states are dynamically feasible
        We assume pure ballistic motion during flight, and the leg angle can be instantaneously controlled
        to any angle (leg angle does not show up in the equations of motion)

        Inputs: state variables
        Outputs: Checks if z_double_dot == 0 -g and x_double_dot ==0
        """

        x_dd = qddot[0]
        z_dd = qddot[1]
        
        return x_dd + z_dd + self.g

    def get_foot_z_position(self, q):
        """
        foot position is not a state variable, but derived from center of mass
        position and leg configuration 

        useful for checking if the foot is in contact with ground for stance phase constraint
        """

        leg_height = q[2]*np.cos(q[3])
        
        return q[1] - leg_height


    def create_contact_schedule(self,trajectory_length):
        """
        Given a trajectory length in # of collocation points, find the
        contact schedule as a binary vector of length trajectory length
        
        0 => flight phase
        1 => stance phase

        This is only done once before the optimization 

        ASSUMED: 
        
            The SLIP model begins a trajectory in the beginning
            of a flight phase

            There are 50 collocation points per contact mode

        Input: trajectory length in collocation points

        Output: contact schedule vector 
        """

        if(trajectory_length % self.collocation_nodes_per_contact != 0):
            raise Exception("trajectory length is not divisible by collocation nodes per contact phase")

        contact_schedule = np.zeros(trajectory_length)

        cnpc = self.collocation_nodes_per_contact
        for idx in range(trajectory_length):

            start, end = (idx*cnpc,idx*cnpc+50)
            #if idx is even, then we are in flight phase
            if idx % 2 == 0:
                contact_schedule[start:end] = 0
            else:
                contact_schedule[start:end] = 1

        return contact_schedule



# Mathematical Program

In [ ]:
prog=MathematicalProgram()

#50 collocation points per contact phase, starts in beginning of flight
number_of_collocation_points = 450

minimum_clearance_height = 0.5 # every jump apex must be above this height

goal_displacement = 10
#slip model initial and final states
#[x,z,l,theta]
#
q_initial_state = np.array([0,3,0,0])
q_dot_initial_state = np.array([0,0,0,0])

q_final_state = np.array([goal_displacement,q_initial_state[1],0,0])
q_dot_final_state = np.array([0,0,0,0])

#constraints for max/min timestep lengths in swing and stance

h_max_swing=.05
h_min_swing=.001

h_max_stance=.05
h_min_stance=.001


## set up decision variables ##

#decide time step lengths for swing and stance
h_swing = prog.AddContinuousVariables(1,"h_swing")
h_stance = prog.AddContinuousVaruiables(1,"h_stance")

#decide state positions 
q=prog.NewContinuousVariables(number_of_collocation_points+1,4,'q')
#decide state velocities
qd=prog.NewContinuousVariables(number_of_collocation_points+1,4,'qd')
#decide state accelerations
qdd=prog.NewContinuousVariables(number_of_collocation_points,4,'qdd')
#decide leg input torques
u=prog.NewContinuousVariables(number_of_collocation_points,1,'u') 

#create SLIP instance
slip_model = SLIP_Model()

ground_schedule = slip_model.create_contact_schedule(number_of_collocation_points)


## set up constraints ## 

#initial state constraints
    #position
prog.AddLinearConstraint( eq( q[0,0], q_initial_state[0] )) #x position
prog.AddLinearConstraint( eq( q[0,1], q_initial_state[1] )) #z position
    #velocity
prog.AddLinearConstraint( eq( qd[0,0], q_dot_initial_state[0] )) #x vel
prog.AddLinearConstraint( eq( qd[0,1], q_dot_initial_state[1] )) #z vel

#final state contraints
    #position
prog.AddLinearConstraint( eq( q[number_of_collocation_points,0], q_final_state[0] )) 
prog.AddLinearConstraint( eq( q[number_of_collocation_points,1], q_final_state[1] )) 
    #velocity
prog.AddLinearConstraint( eq( qd[number_of_collocation_points,0], q_dot_final_state[0] ))
prog.AddLinearConstraint( eq( qd[number_of_collocation_points,1], q_dot_final_state[1] )) 

#Actuator torque bound constraint 
prog.AddBoundingBoxConstraint(lower = [slip_model.min_leg_torq]*number_of_collocation_points, 
                              upper = [slip_model.min_leg_torq]*number_of_collocation_points,
                              vars=u
                              )

# time step constraints 
prog.AddBoundingBoxConstraint(h_min_swing,h_max_swing,h_swing)
prog.AddBoundingBoxConstraint(h_min_stance,h_max_stance,h_stance)

# Contact Sequence Dependent Dynamics Constraints 



## run solver ##

for i in range(number_of_collocation_points):
    #if swing:
        #swing dynamics constraints

    #else:
        #stance dynamics constraints
    pass 







Ground Equations of Motion 

In [19]:
Math(r'\left[\begin{matrix}m \left(- g \sin{\left(\operatorname{t_{1}}{\left(t \right)} \right)} + \operatorname{L_{1}}{\left(t \right)} \frac{d^{2}}{d t^{2}} \operatorname{t_{1}}{\left(t \right)} + 2 \frac{d}{d t} \operatorname{L_{1}}{\left(t \right)} \frac{d}{d t} \operatorname{t_{1}}{\left(t \right)}\right) \operatorname{L_{1}}{\left(t \right)}\\g m \cos{\left(\operatorname{t_{1}}{\left(t \right)} \right)} - k \left(L_{1 0} - \operatorname{L_{1}}{\left(t \right)}\right) - m \operatorname{L_{1}}{\left(t \right)} \left(\frac{d}{d t} \operatorname{t_{1}}{\left(t \right)}\right)^{2} + m \frac{d^{2}}{d t^{2}} \operatorname{L_{1}}{\left(t \right)}\end{matrix}\right]')

<IPython.core.display.Math object>

In [ ]:

##################### I copied this code from online ############# 
#This program works to get the hopper to jump up steps at a steady rate

prog=MathematicalProgram()
N_step=150

step_length=0.5
step_height=0.5

y_i = 0.5 #initial height

q_initial=[0,y_i,0,0,0]
#qd_initial=[0,0,0,0,0]

q_final=[step_length,y_i+step_height,0,0,0]

h_max_air=.05
h_min_air=.001

h_max_g=.05
h_min_g=.001

#decision variables
h_air=prog.NewContinuousVariables(1,'h_air') #time step air
h_g=prog.NewContinuousVariables(1,'h_g') #time step ground

q=prog.NewContinuousVariables(N_step+1,4,'q')
qd=prog.NewContinuousVariables(N_step+1,4,'qd')
qdd=prog.NewContinuousVariables(N_step,4,'qdd')
u=prog.NewContinuousVariables(N_step,1,'u') 

#make grounded variable pre-scheduled 
number_of_contacts=1
grounded_step=get_ground_schedule(N_step,number_of_contacts)
M=100

#sets initial values for x, y, thetas are allowed to move freely
prog.AddLinearConstraint(q[0,0]==q_initial[0])
prog.AddLinearConstraint(q[0,1]==q_initial[1])

#ensures final values are same as initial values for position and velocty, with final x and y shifted by step_height and step_width
prog.AddLinearConstraint(q[N_step,0]==q_final[0])
prog.AddLinearConstraint(q[N_step,1]==q_final[1])
prog.AddLinearConstraint(q[N_step,4]==q_final[4])
prog.AddLinearConstraint(q[N_step,2]==q[0,2])
prog.AddLinearConstraint(q[N_step,3]==q[0,3])
prog.AddLinearConstraint(eq(qd[N_step],qd[0])) 

prog.AddLinearConstraint(qd[0,1]>=0) #helps push solver in correct direction to get it to solve

#actuator constraints
prog.AddBoundingBoxConstraint([0]*N_step,[0.8]*N_step,uZ)
prog.AddBoundingBoxConstraint([-10]*N_step,[10]*N_step,uT)

#time step constraints
prog.AddBoundingBoxConstraint(h_min_air,h_max_air,h_air)
prog.AddBoundingBoxConstraint(h_min_air,h_max_air,h_air_2)
prog.AddBoundingBoxConstraint(h_min_g,h_max_g,h_g)

for t in range(N_step):
  prog.AddBoundingBoxConstraint(-1*np.ones(1),0.1*np.ones(1),q[t+1,4]) #fails with -0.8 as lower bound, even though it never reaches -0.8

  c=(q[t+1,4]+0.5)**2 >= 0.00001
  #print(c)
  prog.AddConstraint(c) #not equal to -0.5

  if grounded_step[t]: #grounded dynamics
    prog.AddLinearConstraint(q[t+1,1]<=0) #constrains toe to be on the ground during scheduled grounded times

    prog.AddConstraint(eq(q[t+1] , q[t] + h_g * qd[t+1]))
    prog.AddConstraint(eq(qd[t+1] , qd[t] + h_g * qdd[t]))

    var=np.concatenate((q[t+1],qd[t+1],qdd[t],uZ[t],uT[t]))
    prog.AddConstraint(manipulator_equations_ground, lb=[0]*nq, ub=[0]*nq, vars=var) #dynamics constraint

    prog.AddCost(h_g[0]*uT[t].dot(uT[t]))
    prog.AddCost(h_g[0]*uZ[t].dot(uZ[t]))

  else: #air dynamics
    prog.AddLinearConstraint(q[t+1,1]>=0) #toe is above ground while in the air
    if t>N_step/2: #selects which time step to use
      #print("ooh")
      h=h_air_2
    else:
      h=h_air
    prog.AddLinearConstraint(eq(uZ[t],0)) #if in air, no spring pre loading. Helps solver find solutions too
    prog.AddConstraint(eq(q[t+1] , q[t] + h * qd[t+1]))
    prog.AddConstraint(eq(qd[t+1] , qd[t] + h * qdd[t]))
    
    var=np.concatenate((q[t+1],qd[t+1],qdd[t],uZ[t],uT[t],))
    prog.AddConstraint(manipulator_equations_air, lb=[0]*nq, ub=[0]*nq, vars=var) #dynamics constraint
    
    prog.AddCost(h[0]*uT[t].dot(uT[t]))
    #prog.AddCost(h[0]*uZ[t].dot(uZ[t])) #not needed because uZ[t] is 0


solver=SnoptSolver()
result=solver.Solve(prog)

print(result.is_success())

q_star_step=result.GetSolution(q)
qd_star_step=result.GetSolution(qd)
qdd_star_step=result.GetSolution(qdd)
h1_step=result.GetSolution(h_air)
h2_step=result.GetSolution(h_g)
h3_step=result.GetSolution(h_air_2)
print(h1_step,h2_step,h3_step)
steps=np.linspace(0,N_step,N_step+1)
uZ_star_step=result.GetSolution(uZ)
uT_star_step=result.GetSolution(uT)
print(GetInfeasibleConstraints(prog,result))

plt.figure()
plt.plot(steps[1:],grounded_step)
plt.plot(steps,q_star_step[:,4])

plt.figure()
plt.plot(steps[1:],uZ_star_step)

plt.figure()
plt.plot(q_star_step[:,0],q_star_step[:,1])